# Diffusion for a Toy SU(2) model

In this notebook, we look at a toy matrix model defined in terms of a single ${\rm SU}(2)$ degree of freedom occupying a single site on a zero dimensional lattice. The toy action we work with is given by $$S[U] = -\frac{\beta}{2} {\rm Re}{\rm Tr}(U),$$ where $\beta$ is a coupling strength.

The diffusion model setup will be entirely Euclidean in the variance-expanding picture. Our approach will be to take pre-generated configurations $U$, project them into the Lie Algebra, and extract the 3 coefficients on the group generators to use as diffusion data.

$$U \xrightarrow{\log} A = \sum_j c_j \sigma_j \xrightarrow{\langle \sigma_j, \cdot\rangle} \left\{c_j\right\}_{j=1, 2, 3}$$
Then we "do" the diffusion (training and sampling in the usual way) to obtain new generator coefficients, then recompose these coefficients into new ${\rm SU}(2)$ matrices:

$$ \left\{c_j'\right\}_{j=1, 2, 3} \xrightarrow{\text{recombine}} \sum_j c_j' \sigma_j = A' \xrightarrow{\text{exponentiate}} \exp(iA') = U'$$

In [ ]:
# General imports
import torch
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
# Imports from our repo
import sys
sys.path.insert(0, '..')  # repo source code

from src.linalg import trace, adjoint
from src.action import SUNToyAction
from src.diffusion import VarianceExpandingDiffusion
from src.sun import (
    proj_to_algebra, 
    random_sun_element,
    group_to_coeffs, coeffs_to_group
)

from src.utils import grab
from src.devices import set_device, get_device, summary

In [ ]:
# Set a device
import src.devices as devices  # from our src code

devices.set_device('cpu')
print(devices.summary())

## Generate training data

For this example, we will try to learn a target density defined by our simple toy action. To learn this density through a diffusion model, we require training data. We will generate training configs by the Metropolis algorithm.

In [ ]:
# Instantiate and test a toy SU(2) matrix action
def _test_action():
    batch_size = 3
    Nc = 2
    U = random_sun_element(batch_size, Nc=2)

    action = SUNToyAction(beta=1.0)
    print('Action evaluated on configs:', grab(action(U)))

_test_action()

In [ ]:
def apply_metropolis(batch_size, Nc, action, num_steps, step_size):
    """Batched Metropolis sampler."""
    action_vals = []
    accept_rates = []
    
    U = random_sun_element(batch_size, Nc=Nc)
    for _ in range(num_steps):
        # Proposal
        V_re = torch.eye(Nc).repeat(batch_size, 1, 1) 
        V_im = step_size * torch.randn((batch_size, Nc, Nc))
        V = V_re + 1j*V_im
        V = torch.matrix_exp(1j*proj_to_algebra(V))  # random group noise
        Up = V @ U
        dS = action(Up) - action(U)

        # Accept / Reject
        r = torch.rand((batch_size,))  # accept w/ prob = exp(-dS)
        accept_mask = (r < torch.exp(-dS))[:, None, None]
        U = torch.where(accept_mask, Up, U)

        action_vals.append(grab(action(U).mean()))
        accept_rates.append(grab(torch.sum(accept_mask) / batch_size))

    return U, action_vals, accept_rates

In [ ]:
# Define physical theory
beta = 3.5
action = SUNToyAction(beta)


# Generate samples
num_train = 2 ** 10
num_therm = 500
num_iters = 1_000
step_size = 0.9

U_train, action_vals, accept_rates = apply_metropolis(
    batch_size = num_train,
    Nc = 2,
    action = action,
    num_steps = num_therm + num_iters,
    step_size = step_size
)


# Visualize Metropolis
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
for ax in axes:
    ax.set_xlabel('Metropolis Steps')

axes[0].plot(action_vals)
axes[0].set_ylabel('Average Action')

axes[1].plot(accept_rates)
axes[1].set_ylabel('Acceptance Rate')

fig.tight_layout()
fig.show();

## Build a score network

First we will build a simple score network that acts on the generator coefficients.

*Note:* There are two score network implementations worth testing: one with and one without time-dependence (see below)

In [ ]:
# First: NO time dependence
class SimpleScoreNetSU2(torch.nn.Module):
    """Time-independent MLP score network for SU(2) generator coefficients."""
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(3, 8),
            torch.nn.SiLU(),
            torch.nn.Linear(8, 8),
            torch.nn.SiLU(),
            torch.nn.Linear(8, 3)
        )

    def forward(self, x, t=None):
        """
        Forward pass. 

        Args:
            x (Tensor): Coefficients of Lie group generators
            t (Tensor, optional): Time step
        """
        del t
        return self.net(x)

        
def _test_su2_score_net():
    score_net = SimpleScoreNetSU2()
    U = random_sun_element(batch_size=5, Nc=2)
    x = group_to_coeffs(U).real
    out = score_net(x)
    assert x.shape == out.shape, \
        '[FAILED: Network output must have same shape as input]'
    print('[PASSED]')


_test_su2_score_net()

In [ ]:
# With time dependence (no embedding)
class TimeDependentScoreNetSU2(torch.nn.Module):
    """Time-dependent MLP score network for SU(2) generator coefficients."""
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(4, 8),  # 3 su2 gens + 1 time
            torch.nn.SiLU(),
            torch.nn.Linear(8, 8),
            torch.nn.SiLU(),
            torch.nn.Linear(8, 3)
        )

    def forward(self, x, t):
        """
        Forward pass. 

        Args:
            x (Tensor): Coefficients of Lie group generators
            t (Tensor): Time step
        """
        t = t.unsqueeze(-1)
        x = torch.cat([x, t], dim=-1)
        return self.net(x)


def _test_su2_score_net():
    score_net = TimeDependentScoreNetSU2()
    U = random_sun_element(batch_size=5, Nc=2)
    x = group_to_coeffs(U).real
    t = torch.rand((x.shape[0],))
    out = score_net(x, t)
    assert x.shape == out.shape, \
        '[FAILED: Network output must have same shape as input]'
    print('[PASSED]')


_test_su2_score_net()

## Train the diffusion model

We will train our score network using the traditional score matching loss function.

In [ ]:
def score_matching_loss(x_0, score_net, diffuser, tol=1e-5):
    t = torch.rand((x_0.shape[0],))
    t = (1 - tol) * t + tol  # avoid endpoints where score can become unstable
    x_t = diffuser(x_0, t)
    
    sigma_t = diffuser.sigma_func(t)[:, None]
    score = score_net(x_t, t) / sigma_t
    eps = (x_t - x_0) / sigma_t
    loss = torch.sum((score*sigma_t + eps)**2, dim=-1)
    return loss.mean()

In [ ]:
# Make diffuser
sigma = 25
diffuser = VarianceExpandingDiffusion(sigma)


# Make a score net
#score_net = SimpleScoreNetSU2()
score_net = TimeDependentScoreNetSU2()


# Training hyperparams
num_epochs = 1_000
lr = 1e-2
optimizer = torch.optim.Adam(params=score_net.parameters(), lr=lr)


# Prepare data
x_0 = group_to_coeffs(U_train).real


# Training loop
losses = []
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    loss = score_matching_loss(x_0, score_net, diffuser)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch {epoch} / {num_epochs} | Loss = {loss.item():.6f}')
    losses.append(loss.item())

In [ ]:
# Plot training loss over time
fig, ax = plt.subplots(1, 1)
ax.plot(losses)
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Loss')
fig.show();

## Sample new generator coefficients from the trained model

Now we will use simple numerical integration to execute the reverse (denoising) process and recover new data samples

In [ ]:
num_samples = 2 ** 10
num_steps = 200

new_action_vals = []
with torch.no_grad():
    # Sample noisy prior data
    prior_sigma = diffuser.sigma_func(t=1.0)
    x_1 = prior_sigma * torch.randn((num_samples, 3))

    # Solve the SDE with Euler-Maruyama
    dt = 1 / num_steps
    times = torch.linspace(1.0, dt, num_steps)
    
    x_t = x_1.clone()
    for i, t in enumerate(times):
        t = t.repeat(num_samples,)
        noise = torch.randn_like(x_0)
        g_t = diffuser.noise_coeff(t)[:, None]
        score = score_net(x_t, t) / diffuser.sigma_func(t)[:, None]
        
        x_t = x_t + g_t**2 * score * dt + g_t * noise * dt**0.5  # Euler-Maruyama
        #x_t = x_t + 0.5 * g_t**2 * score * dt  # Euler
        
        S = action(coeffs_to_group(x_t))
        new_action_vals.append(S.mean().item())
        if i % 10 == 0:
            print(f'Step {i} / {num_steps} | Action = {S.mean().item():.6f}')

## Validation and Visualization

To compare our samples, we will look at the average value of the action on the different batches of configurations as well as the density of the eigenangle $\theta$ where $\lambda = e^{i\theta}$.

In [ ]:
U_prior = coeffs_to_group(x_1)
U_new = coeffs_to_group(x_t)

S_prior = action(U_prior).mean().item()
S_train = action(U_train).mean().item()
S_new = action(U_new).mean().item()
print('Action on U_prior:', S_prior)
print('Action on U_train:', S_train)
print('Action on U_new:', S_new)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_xlabel('Reverse Time Steps')
ax.set_ylabel('Average Action')
ax.plot(new_action_vals, label='Diffusion Samples')
ax.hlines(S_train, xmin=0, xmax=num_steps-1, ls='--', color='black', label='Metropolis Samples')
ax.legend()
fig.show();

In [ ]:
# Training data (Metropolis)
D_train, _ = torch.linalg.eig(U_train)
th_train = torch.angle(D_train)

# Prior data (Gaussian)
D_prior, _ = torch.linalg.eig(U_prior)
th_prior = torch.angle(D_prior)

# Posterior data (Diffusion)
D_new, _ = torch.linalg.eig(U_new)
th_new = torch.angle(D_new)

# plot angular densities
fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
axes[0].set_ylabel('Density')
for i, ax in enumerate(axes):
    ax.hist(grab(th_train[:, i]), bins=50, density=True, alpha=0.65, label='Training (Metropolis)')
    ax.hist(grab(th_prior[:, i]), bins=50, density=True, alpha=0.5, label='Prior (Gaussian)')
    ax.hist(grab(th_new[:, i]), bins=50, density=True, alpha=0.5, label='Diffusion Model')
    ax.set_xlabel(rf'$\theta_{i+1}$')
axes[-1].legend()
fig.tight_layout()
fig.show()